In [1]:
import shutil
from subprocess import check_output
from pathlib import Path
import numpy as np
import os
#import matplotlib.pyplot as plt
import pandas as pd
#from shapely.geometry import Polygon,
import RClib as RClib
import objectdetectionlib as ODlib



In [2]:
%load_ext autoreload
%autoreload 2
''
config = RClib.loadconfigs('.\config_scanner.json')
if Path(config['workspace'] + '/rcprocessingdf.pkl').is_file() and config['use_initialdf'] :
    print('Read scandf from pickle')
    scandf = pd.read_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))
else:
    scandf = RClib.provide_scandf(inputdirectory=config['workspace'], imageformat = config['baseimageformat'])
    
    scandf= scandf.apply(RClib.defineRawTherapeeOutput, foldername='02_TIF', axis=1)
    scandf= scandf.apply(RClib.defineRealityCaptureOutput, foldername='03_RC', axis=1)
    scandf.to_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))

for index, scan in scandf.iterrows():
    print('Processing scan: ' + scan['id'])
    print('Number of images: ' + str(len(scan['imagedf'])))
    #print('Number of masks: ' + str(len(scan['imagedf'][scan['imagedf']['maskimg_path'].notnull()])))



Processing scan: camrun-20210430_084103-w18003ai
Number of images: 504
Processing scan: camrun-20210430_124001-w18003cb
Number of images: 512
Processing scan: camrun-20210503_104034-w18202-19
Number of images: 576
Processing scan: camrun-20210504_070549-w18204-34
Number of images: 449
Processing scan: camrun-20210504_112002-w18206-29
Number of images: 520
Processing scan: camrun-20210505_104833-w18208-54
Number of images: 662
Processing scan: camrun-20210602_123113-w20191
Number of images: 1191
Processing scan: camrun-20210609_101257-w20500
Number of images: 1236
Processing scan: camrun-20210621_114828-w20030-101
Number of images: 1367
Processing scan: camrun-20210624_112406-W31XII60S128
Number of images: 1540
Processing scan: camrun-20210625_110251-w21415-21
Number of images: 2605


In [3]:


scandf2=pd.DataFrame()

for index,scan in scandf.iterrows():
    if not scan['imagedf'] is None and config['developimages']:
        if 'pp3file' in scan.keys() and scan['pp3file']:    
            scan['imagedf'] = scan['imagedf'].apply(RClib.developwithRawTherapee, pp3filepath=scan['pp3file'][0],  outputfolderpath=scan['RToutputfolder'], axis=1)     
            scandf2 = scandf2.append(scan)
    else:
        scan['imagedf'] = scan['imagedf'].apply(RClib.baseimageIsDevimage, axis=1)
        print('Number of images after development: ' + str(len(scan['imagedf'])))
        scandf2 = scandf2.append(scan)

Number of images after development: 504


C:\Users\mhaibt\AppData\Local\Temp\ipykernel_18372\2106716510.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf2 = scandf2.append(scan)


Number of images after development: 512


C:\Users\mhaibt\AppData\Local\Temp\ipykernel_18372\2106716510.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf2 = scandf2.append(scan)


Number of images after development: 576


...

C:\Users\mhaibt\AppData\Local\Temp\ipykernel_18372\2106716510.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf2 = scandf2.append(scan)


Number of images after development: 1540


C:\Users\mhaibt\AppData\Local\Temp\ipykernel_18372\2106716510.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf2 = scandf2.append(scan)


Number of images after development: 2605


C:\Users\mhaibt\AppData\Local\Temp\ipykernel_18372\2106716510.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf2 = scandf2.append(scan)


In [4]:
%load_ext autoreload
%autoreload 2

scandf2a = pd.DataFrame()
for index,scan in scandf2.iterrows():
    print('Number of images before masking: ' + str(len(scan['imagedf'])))
    print('Start masking scan: ' + scan['id'])
    if not scan['imagedf'] is None and config['maskobject_QS3D']:
        scan['imagedf'] = scan['imagedf'].apply(RClib.provide_imageinfo_scanner, axis=1)
        print('Number of images after imageinfo: ' + str(len(scan['imagedf'])))
        scan['imagedf'] = scan['imagedf'].apply(ODlib.maskoutobject_QS3D, axis=1)
        print('Number of images before shotnumber: ' + str(len(scan['imagedf'])))
        scan['imagedf'] = RClib.add_shotnumber(scan['imagedf'] )
        #print(scan['imagedf']['shotnumber'])
        
    else:
        print('no images to mask out')
    print('Number of images after masking: ' + str(len(scan['imagedf'])))
    scandf2a = scandf2a.append(scan)
    #scandf2a = RClib.add_shotnumber(scandf2a)
    #print(group for group in scan['imagedf'].groupby('shotnumber'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Number of images before masking: 504
Start masking scan: camrun-20210430_084103-w18003ai
Number of images after imageinfo: 504
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask 

C:\Users\mhaibt\AppData\Local\Temp\ipykernel_18372\1670280408.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf2a = scandf2a.append(scan)


Number of images after imageinfo: 512
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Ma

C:\Users\mhaibt\AppData\Local\Temp\ipykernel_18372\1670280408.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf2a = scandf2a.append(scan)


Number of images after imageinfo: 576
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Ma

...

C:\Users\mhaibt\AppData\Local\Temp\ipykernel_18372\1670280408.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf2a = scandf2a.append(scan)


Number of images after imageinfo: 1540
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
M

C:\Users\mhaibt\AppData\Local\Temp\ipykernel_18372\1670280408.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf2a = scandf2a.append(scan)


Number of images after imageinfo: 2605
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
Mask already exists
M

C:\Users\mhaibt\AppData\Local\Temp\ipykernel_18372\1670280408.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf2a = scandf2a.append(scan)


In [5]:
%load_ext autoreload
%autoreload 2
config = RClib.loadconfigs('.\config_scanner.json')
scandf_calibparams = pd.DataFrame()
scandf_precalib = scandf2a
if config['calibratecameraparameters']:
    for index, scan in scandf_precalib.iterrows():
        print('Number of images before import: ' + str(len(scan['imagedf'])))
        scan2 = RClib.importRegisteredParameters(scan)
        print('Number of images: ' + str(len(scan2['imagedf'])))
        print('Number of calibrations: ' + str(len(scan2['imagedf'][scan2['imagedf']['f'].notnull()])))
        scandf_calibparams = scandf_calibparams.append(scan2)
    resultparams = RClib.plotParametersStats(scandf_calibparams)
    # save resultparams to json using pandas
    pd.DataFrame(resultparams).to_json(str(Path(config['workspace']) / 'fixCamparams.json'), orient='records')






The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
%load_ext autoreload
%autoreload 2

        
scandf3=pd.DataFrame()
if config['align']:
    for index,scan in scandf2a.iterrows():
        if not scan['imagedf'] is None :    
            scan = RClib.makeImagelist(scan, imagelistname=scan['id'])  
            scan = RClib.makeImagelist(scan, imagelistname= scan['id'] +'_masks' , imagefield='maskimg_path')
            scan = RClib.createRCproject(scan)
            scan =RClib.readProcessingstateFile(scan)    
            if not RClib.checkProcessingstate(scan,command='commandlist_alignsettings'):
                scan['commandlist_alignsettings']=Path(config['alignmentsettings'])
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_alignsettings' )
            if not RClib.checkProcessingstate(scan,command='align-rccmdpath'):
                scan['commandlist_align'] = []
                scan['commandlist_align'].append('-add ' + str(scan['list_dev-img_path']))
                if config['maskobject_QS3D']:
                    scan['commandlist_align'].append('-add ' + str(scan['list_maskimg_path']))
                if 'imageparams' in config and config['imageparams'] == 'constant':
                    scan['commandlist_align'].append('-selectAllImages')
                    scan['commandlist_align'].append('-setFeatureSource 2')
                    scan['commandlist_align'].append('-setPriorLensGroup 1')
                    scan['commandlist_align'].append('-setPriorCalibrationGroup 1')
                if 'imageparams' in config and config['imageparams'] == 'variable':
                    scan['commandlist_align'].append('-selectAllImages')
                    scan['commandlist_align'].append('-setFeatureSource 2')
                    scan['commandlist_align'].append('-setPriorLensGroup -1')
                    scan['commandlist_align'].append('-setPriorCalibrationGroup -1')
                if 'imageparams' in config and config['imageparams'] == 'fixed' and Path(config['imageparamsfile']).is_file():
                    #read imageparams from json-file which is defined in config
                    imageparams = pd.read_json(config['imageparamsfile'], orient='records')
                    #makeImagelist of each group of images with same 'cam_id'
                    # and add to commandlist_align
                    for name, group in scan['imagedf'].groupby('cam_id'):
                        pseudoscan = scan
                        pseudoscan['imagedf'] = group
                        group = RClib.makeImagelist(pseudoscan, imagelistname=scan['id'] + '_' + str(name) + '_images' , imagefield='dev-img_path')
                        scan['commandlist_align'].append('-importImageSelection ' + str(group['list_dev-img_path']))
                        scan['commandlist_align'].append('-editInputSelection "inpCalibration=2"')
                        scan['commandlist_align'].append('-editInputSelection "inpFocal=' + str(imageparams[imageparams['cam_id']== name].iloc[0]['f_mean'])+ '"')
                        scan['commandlist_align'].append('-editInputSelection "inpPPX=' + str(imageparams[imageparams['cam_id']== name].iloc[0]['px_mean'])+ '"')
                        scan['commandlist_align'].append('-editInputSelection "inpPPY=' + str(imageparams[imageparams['cam_id']== name].iloc[0]['py_mean'])+ '"')
                        scan['commandlist_align'].append('-editInputSelection "inpDistortion=0"')
                        scan['commandlist_align'].append('-setPriorLensGroup -1')
                if config['use_gcps']:
                    scan['commandlist_align'].append('-detectMarkers')
                    if Path(config['constantgcps']).is_file() and Path(config['gcpsparamfile']).is_file():
                        scan['commandlist_align'].append('-importGroundControlPoints ' + str(config['constantgcps']) + ' ' + config['gcpsparamfile'])
                    if not Path(config['constantgcps']).is_file() and Path(config['gcpsparamfile']).is_file() and scan['gcpsfile']:
                        scan['commandlist_align'].append('-importGroundControlPoints ' + str(scan['gcpsfile'][0]) + ' ' + config['gcpsparamfile'] )
                
                scan['commandlist_align'].append('-align')
                if config['finealign']:
                    scan['commandlist_align'].append('-selectAllImages')
                    scan['commandlist_align'].append('-setFeatureSource 2')
                    scan['commandlist_align'].append('-setPriorLensGroup -1')
                    scan['commandlist_align'].append('-setPriorCalibrationGroup -1')
                    scan['commandlist_align'].append('-selectMaximalComponent')
                    scan['commandlist_align'].append('-exportXMPForSelectedComponent')
                scan['commandlist_align'].append('-save')
                scan['commandlist_align'].append('-quit')
                scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_align', rccmdpathfield='align-rccmdpath')
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='align-rccmdpath', headless=False )
                RClib.writeProcessingstateFile(scan)
            scandf3 = scandf3.append(scan)
else:
    scandf3 = scandf2a
#pd.write_csv(Path(config['workspace']) / 'rcprocessingdf.csv', sep=';')
#scandf2.to_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
E:\testSAM\nice_tablets\camrun-20210430_084103-w18003ai\03_RC\commandlist_align_camrun-20210430_084103-w18003ai.rccmd


C:\Users\mhaibt\AppData\Local\Temp\ipykernel_18372\1499324562.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf3 = scandf3.append(scan)


E:\testSAM\nice_tablets\camrun-20210430_124001-w18003cb\03_RC\commandlist_align_camrun-20210430_124001-w18003cb.rccmd


C:\Users\mhaibt\AppData\Local\Temp\ipykernel_18372\1499324562.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf3 = scandf3.append(scan)


E:\testSAM\nice_tablets\camrun-20210503_104034-w18202-19\03_RC\commandlist_align_camrun-20210503_104034-w18202-19.rccmd


...

C:\Users\mhaibt\AppData\Local\Temp\ipykernel_18372\1499324562.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf3 = scandf3.append(scan)


E:\testSAM\nice_tablets\camrun-20210624_112406-W31XII60S128\03_RC\commandlist_align_camrun-20210624_112406-W31XII60S128.rccmd


C:\Users\mhaibt\AppData\Local\Temp\ipykernel_18372\1499324562.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf3 = scandf3.append(scan)


E:\testSAM\nice_tablets\camrun-20210625_110251-w21415-21\03_RC\commandlist_align_camrun-20210625_110251-w21415-21.rccmd


C:\Users\mhaibt\AppData\Local\Temp\ipykernel_18372\1499324562.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf3 = scandf3.append(scan)


In [7]:
%load_ext autoreload
%autoreload 2
scandf2= scandf2.apply(RClib.defineResultOutput, foldername='04_GEO', axis=1)
#while 'reconstruct-rccmdpath'.isin(scandf3['processingstate'])] and 'model-rccmdpath'.notin(scandf3['processingstate']) :
    #tobemodeleddf = scandf3['reconstruct-rccmdpath'.isin(scandf3['processingstate']) and 'model-rccmdpath'.notin(scandf3['processingstate']) ]
scandf3 = pd.DataFrame()
if config['reconstruct']:
    for index,scan in scandf2.iterrows():
        if 'constantrcbox' in config.keys() and Path(config['constantrcbox']).is_file():
            scan['rcboxpath'] = Path(config['constantrcbox'])
        else:
            scan['rcboxpath'] = Path(scan['RCoutputfolder'] / 'base.rcbox')
        scan =RClib.readProcessingstateFile(scan)    
        if RClib.checkProcessingstate(scan,command='align-rccmdpath') and scan['rcboxpath'].is_file():
            if not RClib.checkProcessingstate(scan,command='commandlist_reconstructsettings'):
                scan['commandlist_reconstructsettings']=Path(config['reconstructsettings'])
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_reconstructsettings')
            if not RClib.checkProcessingstate(scan,command='reconstruct-rccmdpath'): 
                scan['commandlist_reconstruct'] = []
                scan['commandlist_reconstruct'].append('-setReconstructionRegion ' + str(scan['rcboxpath']))  
                scan['commandlist_reconstruct'].append('-calculateNormalModel')  
                scan['commandlist_reconstruct'].append('-renameSelectedModel ' + scan['id'] + '_highpoly')    
                scan['commandlist_reconstruct'].append('-save')
                scan['commandlist_reconstruct'].append('-quit')  
                scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_reconstruct', rccmdpathfield='reconstruct-rccmdpath')
                print('Starting reconstruction for  ', scan['id'])
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='reconstruct-rccmdpath' )
                RClib.writeProcessingstateFile(scan)
        scandf3 = scandf3.append(scan)
else:
    scandf3 = scandf2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
scandf4 = pd.DataFrame()
if config['texture']:
    for index,scan in scandf3.iterrows():
        scan = RClib.readProcessingstateFile(scan)
        if RClib.checkProcessingstate(scan,command='reconstruct-rccmdpath'):
            if not RClib.checkProcessingstate(scan,command='commandlist_texturingsettings'):
                scan['commandlist_texturingsettings']=Path(config['texturingsettings'])
                print('Texturing settings set for ', scan['id'])
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_texturingsettings')
            if not RClib.checkProcessingstate(scan,command='texturing-rccmdpath'):
                scan['commandlist_texturing'] = []
                scan['commandlist_texturing'].append('-selectModel ' + scan['id'] + '_highpoly') 
                scan['commandlist_texturing'].append('-smooth') 
                scan['commandlist_texturing'].append('-cleanModel')
                scan['commandlist_texturing'].append('-renameSelectedModel ' + scan['id'] + '_highpoly' + '_smoothed')
                scan['commandlist_texturing'].append('-correctColors geometry') 
                scan['commandlist_texturing'].append('-unwrap')  
                scan['commandlist_texturing'].append('-calculateTexture')
                scan['commandlist_texturing'].append('-simplify 10000000')
                scan['commandlist_texturing'].append('-cleanModel')
                scan['commandlist_texturing'].append('-renameSelectedModel ' + scan['id'] + '_10M')
                scan['commandlist_texturing'].append('-unwrap')
                scan['commandlist_texturing'].append('-reprojectTexture ' + scan['id'] + '_highpoly' + '_smoothed' + ' ' + scan['id'] + '_10M')
                scan['modelspaths'] = {'modelpath_10M': scan['Resultoutputfolder'] / Path(config['exportname_prefix'] + scan['id'] + '_10M.obj')}
                print(scan['modelspaths'])
                scan['commandlist_texturing'].append('-exportModel ' + scan['id'] + '_10M ' + str(scan['modelspaths'].get('modelpath_10M')) + ' ' + config['exportsettings1'] )
                scan['commandlist_texturing'].append('-selectModel ' + scan['id'] + '_highpoly' + '_smoothed')   
                scan['commandlist_texturing'].append('-simplify 40000000')
                scan['commandlist_texturing'].append('-cleanModel') 
                scan['commandlist_texturing'].append('-renameSelectedModel ' + scan['id'] + '_40M')
                scan['commandlist_texturing'].append('-unwrap')
                scan['commandlist_texturing'].append('-reprojectTexture ' + scan['id'] + '_highpoly' + '_smoothed' + ' ' + scan['id'] + '_40M')
                scan['modelspaths']['modelpath_40M'] = scan['Resultoutputfolder'] / Path( config['exportname_prefix'] + scan['id'] + '_40M.obj')
                scan['commandlist_texturing'].append('-exportModel ' + scan['id'] + '_40M ' + str(scan['modelspaths'].get('modelpath_40M')) + ' ' + config['exportsettings1'] )           
                scan['commandlist_texturing'].append('-save')
                scan['commandlist_texturing'].append('-quit')  
                scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_texturing', rccmdpathfield='texturing-rccmdpath')
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='texturing-rccmdpath' )
                RClib.writeProcessingstateFile(scan)
        scandf4 = scandf4.append(scan)
else:
    scandf4=scandf3
scandf4.to_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))

In [9]:
if config['orthomosaic']:
    for index,scan in scandf3.iterrows():
        scan = RClib.readProcessingstateFile(scan)
        if RClib.checkProcessingstate(scan,command='reconstruct-rccmdpath'):
            if not RClib.checkProcessingstate(scan,command='commandlist_orthoimage'):
                if 'constantorthobox' in config.keys() and Path(config['constantorthobox']).is_file():
                    scan['orthoboxpath'] = Path(config['constantorthobox'])
                    scan['commandlist_orthoimage'] = []
                    scan['commandlist_orthoimage'].append('-selectModel ' + scan['id'] + '_highpoly') 
                    scan['commandlist_orthoimage'].append('-calculateOrthoProjection ' + scan['orthoboxpath'] ) 
                    scan['commandlist_orthoimage'].append('-exportOrthoProjection ' + scan['orthoboxpath'] ) 
            

In [10]:
%load_ext autoreload
%autoreload 2
if config['orthobox2gis']:
    for index,scan in scandf.iterrows():
        #print(scan.keys())
        RClib.read_rcorthobox(scan)
        #print(scan['orthobox'])
        scan['orthobox'].to_file(config['workspace'] + '/Orthoboxes_utm38.shp', driver='ESRI Shapefile')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


KeyError: 'orthobox2gis'